# MDF data analysis

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from load_data import train, test, BIG, TRAIN, TEST, SOURCE
from common.visualization_helper import display_groups, display_unique_count
from common.preprocessing_helper import get_unique_noconst, split_series_values, get_unique_col_values, get_numeric_nonnumeric_cols

In [ ]:
print BIG.shape, BIG.columns.values
print train.shape, train.columns.values
print test.shape, test.columns.values


In [ ]:
import re

In [ ]:
vals = BIG.ix[:, 6].values
for t in vals:
    m = re.match(r'^([\d,]*)\s*([\w\-]+)\D*([\d,]*)\s*([\w\-]*)\D*([\d,]*)\s*([\w\-]*)', t)
    if m is None:
        raise Exception("Not found : %s" % t)

In [ ]:
MEASURE_TYPES = [u'UI', u'mg', u'ml', u'g', u'kg', u'litre']

test = u'litre2'

if test in MEASURE_TYPES:
    print True
else:
    print False

In [ ]:
vals = BIG.ix[:, 6].values


def debug(groups, t):
    if len(groups) == 1:
        if groups[0][1] == 'UI' or groups[0][1] == 'G' or groups[0][1] == 'lyophilisat':
            print "\n\n >>>>", groups
            print ">>>>", i, " : ", t

    elif len(groups) == 2:
        if groups[0][1] == 'ml':
            print "\n\n >>>>", groups
            print ">>>>", i, " : ", t
        if groups[1][1] == 'UI' or groups[1][1] == 'G' or groups[1][1] == 'lyophilisat':
            print "\n\n >>>>", groups
            print ">>>>", i, " : ", t
    else:
        print groups
        print i, " : ", t    
        
        

# First pass : get all types of packages, quantity types etc 
nonnum_types = []
data = []
MEASURE_TYPES = [u'UI', u'mg', u'ml', u'g', u'kg', u'litre']
NON_QUANTITY_TYPES = []
QUANTITY_TYPES = []
for i, t in enumerate(vals):
    tt = t
    ## Add '1' before package type 
    if not tt[0].isdigit():
        tt = '1 ' + tt   
    
    ## Add 1 between 'avec' and a word after
    groups = re.findall("(avec) ([a-zA-Z]+)", t)
    for g in groups:
        init = ' '.join(g)
        repl = '%s 1 %s' % g
        tt = tt.replace(init, repl)
    
    ## Get groups (digits, type)
    groups = re.findall(r'(\d+,?\d*)\s+([\w\-]+)', tt)
    if len(groups) > 0:
        
        for j, g in enumerate(groups):
            if g[1] == 'securite' or g[1] == 'bouchon' or g[1] == 'feuille':
                print groups
                print i, " : ", t

            nonnum_types.append(g[1])
            # Get quantity type : [(digits, quantity_type), (digits, MEASURE_TYPE)]
            if j > 0 and g[1] in MEASURE_TYPES:
#                 if "aiguille" == groups[j-1][1]:
#                     print groups
#                     print "add : ", groups[j-1][1]
#                     print i, " : ", t
                QUANTITY_TYPES.append(groups[j-1][1])
        

        data.append(groups)
    else:
        raise Exception("Not found : %s" % t)
        
nonnum_types = np.unique(np.array(nonnum_types))

print "\n\n nonnum_types : ", len(nonnum_types)
for t in nonnum_types:
    print t

QUANTITY_TYPES = np.unique(np.array(QUANTITY_TYPES))
print "\n\n QUANTITY_TYPES : ", len(QUANTITY_TYPES)
for t in QUANTITY_TYPES:
    print t
    
    

In [ ]:
BIG.ix[8512, 6]

In [ ]:
NONNUM_TYPES = nonnum_types

# Second pass : 
processed_libelles = np.zeros((len(data), len(NONNUM_TYPES)))
PROCESSED_LIBELLES = pd.DataFrame(processed_libelles, columns=NONNUM_TYPES)    
for i, groups in enumerate(data):
    for g in groups:
        PROCESSED_LIBELLES.ix[i, g[1]] = g[0]

PROCESSED_LIBELLES.head()

In [ ]:
PROCESSED_LIBELLES['plaquette'].loc[1]

### Remove some useless columns

In [ ]:
df__noid_noprix = BIG.drop(['libelle', 'prix', 'id'], axis=1)

Check for dupicate rows and lets look at the columns with only one unique value.

In [ ]:
df__noid_noprix_unique_noconst = get_unique_noconst(df__noid_noprix, verbose=True)

### Columns description : 

In [ ]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(df__noid_noprix_unique_noconst)

In [ ]:
display_groups(type_groups)

### Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [ ]:
num_cols, nonnum_cols = get_numeric_nonnumeric_cols(df__noid_noprix_unique_noconst)
        
nb_isnan = np.sum(df__noid_noprix_unique_noconst[num_cols].apply(np.isnan))

In [ ]:
print nb_isnan

### Identify non numeric rows.

In [ ]:
df__noid_noprix_unique_noconst[nonnum_cols].tail(5)

In [ ]:
train__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TRAIN]
train_unique_col_values = get_unique_col_values(train__noid_noprix_unique_noconst[nonnum_cols])
#train_unique_col_values

In [ ]:
test__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TEST]
test_unique_col_values = get_unique_col_values(test__noid_noprix_unique_noconst[nonnum_cols])
#test_unique_col_values

In [ ]:
index = 1
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print "\n Values present in test and not present in train"
non_trained_values = test_df[~test_df['Value'].isin(train_df['Value'])]
print len(non_trained_values),"\n", non_trained_values

print "\n Values present in train and not present in test"
non_test_values = train_df[~train_df['Value'].isin(test_df['Value'])]
print len(non_test_values),"\n", non_test_values

In [ ]:
index = 2
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print "\n Values present in test and not present in train"
non_trained_values = test_df[~test_df['Value'].isin(train_df['Value'])]
print len(non_trained_values),"\n", non_trained_values

print "\n Values present in train and not present in test"
non_test_values = train_df[~train_df['Value'].isin(test_df['Value'])]
print len(non_test_values),"\n", non_test_values

In [ ]:
index = 4
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print "\n Values present in test and not present in train"
non_trained_values = test_df[~test_df['Value'].isin(train_df['Value'])]
print len(non_trained_values),"\n", non_trained_values

print "\n Values present in train and not present in test"
non_test_values = train_df[~train_df['Value'].isin(test_df['Value'])]
print len(non_test_values),"\n", non_test_values

In [ ]:
index = 5
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print "\n Values present in test and not present in train"
non_trained_values = test_df[~test_df['Value'].isin(train_df['Value'])]
print len(non_trained_values),"\n", non_trained_values

print "\n Values present in train and not present in test"
non_test_values = train_df[~train_df['Value'].isin(test_df['Value'])]
print len(non_test_values),"\n", non_test_values

In [ ]:
def alpha_sort(s):
    splt = s.split(' ')
    sorted_splt = sorted(splt)
    return ' '.join(sorted_splt)    

In [ ]:
index = 6
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])
train_res = train_res.apply(alpha_sort)
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_res = test_res.apply(alpha_sort)
test_df = display_unique_count(test_res)

print "\n Values present in test and not present in train"
non_trained_values = test_df[~test_df['Value'].isin(train_df['Value'])]
print len(non_trained_values),"\n", non_trained_values

print "\n Values present in train and not present in test"
non_test_values = train_df[~train_df['Value'].isin(test_df['Value'])]
print len(non_test_values),"\n", non_test_values

In [ ]:
index = 7
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 8
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 9
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 10
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

### Encode categorical features

In [ ]:
df = df__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    df__noid_noprix_unique_noconst[c] = le.transform(df[c])
df__noid_noprix_unique_noconst.head(5)

### Get dummies for 'Substances' and 'Voies admin'

In [ ]:
df = df__noid_noprix_unique_noconst[['substances', 'voies admin']]

def alpha_sort(s):
    splt = s.split(' ')
    sorted_splt = sorted(splt)
    return ' '.join(sorted_splt) 

# Substances : 
def alpha_sort(s):
    splt = s.split(' ')
    sorted_splt = sorted(splt)
    return ' '.join(sorted_splt) 

def alpha_sort_notsplitted(line):
    splt = line.split(',')
    for i in range(len(splt)):
        splt[i] = alpha_sort(splt[i])
    return ','.join(splt) 

df_substances = df['substances'].str.strip().str.replace(', ', ',').apply(alpha_sort_notsplitted).str.get_dummies(',')

# Voies admin : 
dums_va = df['voies admin'].str.get_dummies(',')

In [ ]:
df2 = df__noid_noprix_unique_noconst.drop(['substances', 'voies admin'], axis=1)
df__noid_noprix_unique_noconst = pd.concat([df2, dums_va], axis=1)

In [ ]:
df__noid_noprix_unique_noconst.head(5)

In [ ]:
df_substances.head(3)

### Price histogram

In [ ]:
plt.figure(figsize=(18, 6))
train['logprix'] = train['prix'].apply(np.log)
sns.distplot(train['logprix'], color='green', kde=False)